In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, RobustScaler

from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from imblearn.over_sampling import SMOTE

import joblib

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
#log-transform
def log_transform(X, columns):
    X = X.copy()
    for col in columns:
        X[col] = np.log(X[col] + 1e-8)
    return X
def update_column_names(df):
    df = df.rename(columns={
        'Temperature': 'Temperatura',
        'Humidity': 'Umidade',
        'Wind Speed': 'Velocidade_do_Vento',
        'Cloud Cover': 'Cobertura_de_Nuvens',
        'Pressure': 'Pressao',
        'Location': 'Localizacao',
        'Rain Tomorrow': 'Chovera_Amanha',
        'Precipitation': 'Precipitacao',
        'Date':'Data',
    })
    return df
def get_season(date):
        month = date.month
        day = date.day
        if (((month == 3 and day > 19) and (month == 6 and day < 21)) or (month == 4) or (month == 5)):
                return 'Primavera'
        elif (month == 6 or month == 7 or month == 8 or (month == 9 and day < 23)):
                return 'Verão'
        elif (month == 9 or month == 10 or month == 11 or (month == 12 and day < 21)):
                return 'Outono'
        else:
                return 'Inverno'
# Mapeamento completo de cidades para estados
cidade_para_estado = {
    'Los Angeles': 'California', 
    'Chicago': 'Illinois', 
    'Houston': 'Texas',
    'Phoenix': 'Arizona',
    'Philadelphia': 'Pennsylvania',
    'San Antonio': 'Texas', 
    'San Diego': 'California',
    'Dallas': 'Texas',
    'San Jose': 'California', 
    'Austin': 'Texas', 
    'Jacksonville': 'Florida', 
    'Fort Worth': 'Texas',
    'Columbus': 'Ohio', 
    'Indianapolis': 'Indiana', 
    'Charlotte': 'North Carolina', 
    'San Francisco': 'California',
    'Seattle': 'Washington', 
    'Denver': 'Colorado', 
    'New York': 'New York', 
    'Washington D.C.': 
    'District of Columbia'
}
estado_para_regiao = {
    'California': 'Oeste',
    'Illinois': 'Meio-Oeste',
    'Texas': 'Sul',
    'Arizona': 'Oeste',
    'Pennsylvania': 'Nordeste',
    'Florida': 'Sul',
    'Ohio': 'Meio-Oeste',
    'Indiana': 'Meio-Oeste',
    'North Carolina': 'Sul',
    'Washington': 'Oeste',
    'Colorado': 'Oeste',
    'New York': 'Nordeste',
    'District of Columbia': 'Sul'
}
def cidade_para_regiao(cidade: pd.Series) -> str:
    cidade = cidade.title()
    estado = cidade_para_estado.get(cidade)
    return estado_para_regiao.get(estado) if estado else None

def pre_processar_df(df, numeric_columns):
    # Transformando os dados numéricos para o log
    df = log_transform(df, numeric_columns)

    # Cria a coluna Estacao
    df['Data'] = pd.to_datetime(df['Data'])
    
    # Cria a coluna Estacao
    df = df.sort_values(by='Data', ascending=True)
    
    df["Estacao"] = df["Data"].apply(get_season)
    
    # Cria a coluna Regiao
    df['Regiao'] = df['Localizacao'].apply(cidade_para_regiao)

    # Remove colunas originais que não serão mais usadas
    # df = df.drop(['Data', 'Localizacao'], axis=1)
    return df

In [26]:





def main():
    # Read the CSV file into a DataFrame
    df = pd.read_csv('usa_rain_prediction_dataset_2024_2025.csv')

    # Display the first few rows of the DataFrame
    print(df.head())
    
    df = update_column_names(df)
    
    print(df.head())
    
    df = pre_processar_df(df, ['Temperatura', 'Umidade', 'Velocidade_do_Vento', 'Cobertura_de_Nuvens', 'Pressao', 'Precipitacao'])
    print(df.head())
    
    sc = RobustScaler()
    sc.fit_transform(df[['Temperatura', 'Umidade', 'Velocidade_do_Vento', 'Cobertura_de_Nuvens', 'Pressao', 'Precipitacao']])
    
    df.to_csv('usa_rain_prediction_dataset_2024_2025_preprocessed.csv', index=True)
    

main()

         Date  Location  Temperature   Humidity  Wind Speed  Precipitation  \
0  2024-01-01  New York    87.524795  75.655455   28.379506       0.000000   
1  2024-01-02  New York    83.259325  28.712617   12.436433       0.526995   
2  2024-01-03  New York    80.943050  64.740043   14.184831       0.916884   
3  2024-01-04  New York    78.097552  59.738984   19.444029       0.094134   
4  2024-01-05  New York    37.059963  34.766784    3.689661       1.361272   

   Cloud Cover     Pressure  Rain Tomorrow  
0    69.617966  1026.030278              0  
1    41.606048   995.962065              0  
2    77.364763   980.796739              1  
3    52.541196   979.012163              0  
4    85.584000  1031.790859              0  
         Data Localizacao  Temperatura    Umidade  Velocidade_do_Vento  \
0  2024-01-01    New York    87.524795  75.655455            28.379506   
1  2024-01-02    New York    83.259325  28.712617            12.436433   
2  2024-01-03    New York    80.943050 